# SEIRS model with Cost Analysis

### TODOS:

* Update Betas in list based on infection chance of the occupation

## 0. File Setup
Start here if `worker_data_final.pickle` is not in the file system

In [1]:
import pandas as pd
from seirsplus.models import *
import networkx
import pickle

In [20]:
# build pandas dataframe from excel file
filePath = 'Work_Context.xlsx'
worker_data = pd.read_excel(filePath)
# Save file so it does not have to be recompiled
with open('pickles/worker_data.pickle', 'wb') as file:
    pickle.dump(worker_data, file)

In [34]:
filePath = 'national_M2019_dl.xlsx'
occupation_data = pd.read_excel(filePath)
# Save file so it does not have to be recompiled
with open('pickles/occupation_data.pickle', 'wb') as file:
    pickle.dump(occupation_data, file)

### Start here

In [21]:
with open('pickles/occupation_data.pickle', 'rb') as file:
    occupation_data = pickle.load(file)
with open('pickles/worker_data.pickle', 'rb') as file:
    worker_data = pickle.load(file)

## 1. Parse Data (Dept. of Labor)

The Depertment of Labor data contains a whole swath of contextual information about each occupation within its database. The only features were are interested in are the metrics on a job's `Exposed to Disease or Infections` and `Physical Proximity` measures. These can be found in the `worker_data` object in this notebook and here (https://www.onetcenter.org/database.html#individual-files). `Employment` and `Annual Income` can be found in the `occupation_data` object and this website (https://www.bls.gov/oes/current/oes_nat.htm#11-0000).

In [25]:
""" WORKER_DATA Filter rows based on relevant characteristics """
relevant_elements = ["Exposed to Disease or Infections", "Physical Proximity"]
characteristics = worker_data['Element Name']

worker_data_filtered = worker_data.loc[
    (worker_data['Element Name'].isin(relevant_elements)) # filter on characteristics
    & (worker_data['Scale ID'] == 'CX')] # filter on score (out of 5)

worker_data_filtered = worker_data_filtered.filter(items=['O*NET-SOC Code','Title', 'Element Name', 'Data Value'])
worker_data_filtered["O*NET-SOC Code"] = worker_data_filtered["O*NET-SOC Code"].apply(lambda x: x.split(".")[0])

In [26]:
# Build crosslisting between worker_data and occupation_data based on occ_code
occupation_list = worker_data_filtered['O*NET-SOC Code']
# Strip CC in AAA-BBBB.CC format of the job identifier in the worker_data Dataframe

In [27]:
""" OCCUPATION DATA Filter rows based on relevant characteristics """
# Get population and income data for occupations
occupation_data_filtered = occupation_data.loc[
    (occupation_data['occ_code'].isin(occupation_list))] # filter on characteristics

occupation_data_filtered = occupation_data_filtered.filter(items=['occ_code','occ_title','tot_emp','a_mean'])
occupation_data_filtered = occupation_data_filtered.rename(columns={'occ_title':'Title', 'tot_emp':'Employment','a_mean':'Annual Income', 'occ_code':'O*NET-SOC Code'})

# general_population = occupation_data_filtered.iloc[0][2] # first entry of the Employment column

In [62]:
occupation_data_filtered.head()

,O*NET-SOC Code,Title,Employment,Annual Income
4,11-1011,Chief Executives,205890,193850
6,11-1021,General and Operations Managers,2400280,123030
11,11-2011,Advertising and Promotions Managers,25100,141890
13,11-2021,Marketing Managers,263680,149200
14,11-2022,Sales Managers,402600,141690


In [33]:
worker_data_filtered.head()

,O*NET-SOC Code,Title,Element Name,Data Value
120,11-1011,Chief Executives,Physical Proximity,2.73
168,11-1011,Chief Executives,Exposed to Disease or Infections,1.72
458,11-1011,Chief Sustainability Officers,Physical Proximity,2.92
506,11-1011,Chief Sustainability Officers,Exposed to Disease or Infections,1.12
796,11-1021,General and Operations Managers,Physical Proximity,3.21


In [ ]:
# Only include professions overlap between the 2 tables (1-1)

worker_data_filtered.head()
# merged_inner = pd.merge(left=survey_sub, right=species_sub, left_on='species_id', right_on='species_id')

In [55]:
""" Reformat Dataframe to include the below features """
# Title, Context Score 1, Context Score 2
dataframes = []
for element in relevant_elements:
    df = worker_data_filtered.copy()
    df = df[characteristics == element]
    df = df.drop(columns=['Element Name'])
    df = df.rename(columns={'Data Value': element})
    dataframes.append(df)

from functools import reduce
worker_data_final = reduce(lambda df1,df2: pd.merge(df1,df2,on=['Title', 'O*NET-SOC Code'], how='left'), dataframes)

# Merge with Population data
# worker_data_final = pd.merge(worker_data_final, )
worker_data_final = pd.merge(left=worker_data_final, right=occupation_data_filtered, left_on=['Title','O*NET-SOC Code'], right_on=['Title','O*NET-SOC Code'])
worker_data_final = worker_data_final.drop(columns=['O*NET-SOC Code'])
worker_data_final['Employment'] =  worker_data_final['Employment'].apply(lambda a: int(a)) 
#  worker_data_final['Annual Income'] = worker_data_final['Annual Income'].apply(lambda a: int(a))
worker_data_final.to_csv('worker_final.csv')

In [13]:
general_population = worker_data_final['Employment'].sum()
general_population

101259780

In [7]:
# Save file so it does not have to be recompiled
with open('pickles/worker_data_final.pickle', 'wb') as file:
    pickle.dump(worker_data_final, file)

## 2. Barabasi-Albert Graph generation

Barabasi-Albert Graphs are 

A unique graph will be generated for each occupation listed by the Department of Labor. The number of nodes (`n`) will be determined by the number of individuals of that occupation in the population. The number of connections within the graph (`m`) will be determined by one's physical proximity to others in the workplace.

In [9]:
# Open the pickle of the relevant data 
with open('pickles/worker_data_final.pickle', 'rb') as file:
    worker_data_final = pickle.load(file)

In [4]:
total_population = 10000
avg_interactions_p_day = 17 # (https://www.researchgate.net/figure/Daily-average-number-of-contacts-per-person-in-age-group-j-The-average-number-of_fig2_228649013)

# general_population = 155760000 # 155.76 milion employed individuals in the US

In [11]:
worker_data_final.head()

,Title,Exposed to Disease or Infections,Physical Proximity,Employment,Annual Income
0,Chief Executives,1.72,2.73,205890,193850
1,General and Operations Managers,1.56,3.21,2400280,123030
2,Advertising and Promotions Managers,1.03,2.82,25100,141890
3,Marketing Managers,1.07,2.89,263680,149200
4,Sales Managers,1.13,2.91,402600,141690


In [14]:
# Build a graph for each occupation 

# determine 'n' based on population data
listOfNodes = [] # a list of all the nodes across all graphs
numPasses = 0
for _, row in worker_data_final.iterrows():
    title        = row['Title']
    numNodes     = int((row['Employment'] / general_population)*total_population)
    interaction  = int(row['Physical Proximity']) # TODO: Determine an effective scaling factor -> range(0, 5)
    # print(numNodes, interaction)
    if numNodes <= interaction:
        continue
    baseGraph    = networkx.barabasi_albert_graph(n=numNodes, m=interaction)
    for node in baseGraph.nodes:
        listOfNodes.append((title, len([n for n in baseGraph[node]])))
    numPasses+=1
    # print(numPasses)

In [15]:
len(listOfNodes)

9441

In [16]:
with open('pickles/listOfNodes.pickle', "wb") as file:
    pickle.dump(listOfNodes, file)

## 3. Merging Barabasi-Albert Graphs

This mechanism of merging the Barabasi Graphs of different occupations follows the research here (https://www.researchgate.net/publication/271200973_On_Merging_and_Dividing_of_Barabasi-Albert-Graphs)

Below is an implementation of the `Node-Degree-Order Merge` which runs in `O(n*log(n))`.

This method orders the nodes by the number of connections they contain across all of the graphs being merged. A new graph is then developed using the Barabasi-Albert model by inserting these nodes in order. The analogy this paper uses to this scenario is like a "student graduating from school and going to university". In our model, the number of connections is directly correlated with the physical proximity to others. This merging technique is a mechanism to more effectively scale the physical proximity metric proportionally across occupations. The "graduation" can be considered a transition of popularity (in one's number of interactions) from within an occupation, to a larger population.

The aggregate Barabsi-Albert graph will have parameters reflecting the total population. The number of nodes (`n`) will be the total population relative to the populations of each occupation. The number of connections per iteration (`m`) will be relative to the average number of interactions a given individual will have in a day. Nodes are 0 indexed within `listOfNodes`, so each individual (and their occupation) is easy to track within the system.

In [1]:
# listOfNodes

In [5]:
with open('pickles/listOfNodes.pickle', "rb") as file:
    listOfNodes = pickle.load(file)

In [6]:
listOfNodes.sort(key=lambda x: x[1], reverse=True) # list of Nodes in descending order, sorted by number of connections

full_graph    = networkx.barabasi_albert_graph(n=len(listOfNodes), m=avg_interactions_p_day)
""" listOfNodes and full_graph.nodes has a 1-1 translation """

' listOfNodes and full_graph.nodes has a 1-1 translation '

## 4. Integration with SEIRS

One factor that we have not taken into account yet is the exposure that one has to the disease (Covid-19). Within the SEIRS model, this directly corresponds to the `beta` or `rate of transmission` parameter. We can map a rate of transmission to each node in our Barabasi graph and dynamically modify them in simulations in future sections.

In [7]:
G_normal     = custom_exponential_graph(full_graph, scale=100)
# Social distancing interactions:
G_distancing = custom_exponential_graph(full_graph, scale=10)
# Quarantine interactions:
G_quarantine = custom_exponential_graph(full_graph, scale=5)

In [11]:

betas = [] # TODO: set up betas for each occupation
for individual in listOfNodes:
    title, _        = individual
    infection       = worker_data_final.loc[worker_data_final['Title'] == title]['Exposed to Disease or Infections']
    betas.append(infection*0.1)

In [12]:
model = SEIRSNetworkModel(G=G_normal, beta=0.155, sigma=1/5.2, gamma=1/12.39, mu_I=0.0004, p=0.5,
                          Q=G_quarantine, beta_D=0.155, sigma_D=1/5.2, gamma_D=1/12.39, mu_D=0.0004,
                          theta_E=0.02, theta_I=0.02, phi_E=0.2, phi_I=0.2, psi_E=1.0, psi_I=1.0, q=0.5,
                          initI=10)
                          
with open("pickles/SEIRS_model.pickle", "wb") as file:
    pickle.dump(model, file)

## 5. Running Simulations

In [13]:
with open('pickles/worker_data_final.pickle', 'rb') as file:
    worker_data_final = pickle.load(file)

In [14]:
with open('pickles/listOfNodes.pickle', "rb") as file:
    listOfNodes = pickle.load(file)

In [15]:
class Individual():
    def __init__(self, id, occupation, income, numConn):
        self.id = id
        self.occupation = occupation
        self.income = income
        self.savings = 0.2*income
        self.number_connections = numConn
        self.connections = [] # list of individuals with an active connection
        self.pendingActiveConnection = 0 # pending number of connections the individual is attempting to have
    def payments(self, amount):
        self.income = int(self.income)
        if self.income == '*':
            return
        self.income += amount
    def cost(self, amount):
        self.income = int(self.income)
        if self.income == '*':
            return
        self.income -= amount

In [19]:
def initIndividuals():
    listOfIndividuals = [Individual(id, occupation, worker_data_final.loc[worker_data_final['Title'] == occupation]['Annual Income'],  numConn) for id, (occupation, numConn) in enumerate(listOfNodes)]
    print("Set up the list!")
    for edge in list(model.G.edges):
        a, b = edge
        listOfIndividuals[a].connections.append(b)
        listOfIndividuals[b].connections.append(a)
    print("Set up the edge connections!")

In [20]:
initIndividuals()

Set up the list!
Set up the edge connections!


In [18]:
with open("pickles/individuals.pickle", "wb") as file:
    pickle.dump(listOfIndividuals, file)

NameError: name 'listOfIndividuals' is not defined

In [35]:
with open("pickles/individuals.pickle", "rb") as file:
    listOfIndividuals = pickle.load(file)

In [23]:
with open("pickles/SEIRS_model.pickle", "rb") as file:
    model = pickle.load(file)

In [51]:
"""
Format of @param commands positive or negative values
Protocol: Social distancing is two way (ie. Interactions will only occur if both agree to it). If one does not agree to it, then it will not occur. 
"""
def edgeOperation(graph, individuals, commands):
    nodeList = list(graph.nodes)
    for id, command in enumerate(commands):
        individual = individuals[id]
        if command > 0: # add edges
            for other in individual.connections:
                if other not in nodeList: # node has been removed from the graph
                    continue
                if other.pendingActiveConnections > 0:
                    graph.add
            pass
        if command < 0: # remove edges
            pass
    return graph

    
"""
@returns : percentage increase 
"""    
def incomefunction(income):
    pass

"""
Purpose: Do an action on a set of relevant individuals. 
@returns : a new version of the graph
"""
def stimulusCheck(graph):
    pass

def getLaidOff(graph):
    pass


simulations = [stimulusCheck, getLaidOff]

"""
@param action       : This is the function that will be run every iteration on the graph
@param iterations   : The number of iterations the simulation will be run for
@param unit_time    : The amount of time each iteration will run for
"""
def runSimulation(action, iterations, unit_time): # this action will be taken every round
    with open("pickles/SEIRS_model.pickle", "rb") as file:
        model = pickle.load(file)

    graph = model.G
    for _ in range(iterations):
        graph = action(graph)
        model.update_G(graph) # update the graph with modified edges
        model.run(T=unit_time)

edges = model.G.edges([0]) # constant time
edges
# variable 

TypeError: 'EdgeDataView' object is not subscriptable

t = 0.63
t = 10.10


True